In [1]:
import os
import numpy as np

# defining global variable path
image_path = "./images_training_rev1/new_images/"

#function to load folder into arrays and then it returns that same array'''
def loadImages(path):
    # Put files into lists and return them as one list of size 4
    image_files = sorted([os.path.join(path, file)
         for file in os.listdir(path) if file.endswith('.jpg')])
 
    return image_files

In [2]:
data_img = loadImages(image_path)

In [3]:
import cv2
img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data_img[:500]]
print(img[0].shape)

(64, 64, 3)


In [4]:
img = np.asarray(img)

In [5]:
import pandas as pd
data = pd.read_csv('class1_df.csv', index_col=0)
data = data[:500]

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(img, data, test_size=.2, random_state=15)
print(X_train.shape)
print(y_train.shape)

(400, 64, 64, 3)
(400, 3)


In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform', activation='relu'):
    # create model
    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=(3,3),input_shape=(64,64,3),activation=activation))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=8, kernel_size=(3,3),activation=activation)) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
 
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adamax']
init = ['glorot_uniform', 'normal']
epochs = [50, 100, 500]
batches = [64, 400]
activation = ['softmax', 'relu', 'sigmoid']
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


C:\Users\utilisateur\Anaconda3\envs\computer_vision\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










C:\Users\utilisateur\Anaconda3\envs\computer_vision\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.572500 using {'activation': 'relu', 'batch_size': 64, 'epochs': 100, 'init': 'normal', 'optimizer': 'rmsprop'}
0.225000 (0.131944) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.310000 (0.048193) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'adamax'}
0.380000 (0.114638) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.375000 (0.081455) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 50, 'init': 'normal', 'optimizer': 'adamax'}
0.327500 (0.221337) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.357500 (0.035738) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adamax'}
0.410000 (0.027593) with: {'activation': 'softmax', 'batch_size': 64, 'epochs': 100, 'init': '